<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Group Chat with Retrieval Augmented Generation

AutoGen supports conversable agents powered by LLMs, tools or humans, performing tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [1]:
%%capture --no-stderr
# %pip install pyautogen[retrievechat]~=0.1.11

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [2]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-35-turbo", "gpt-4", "gpt4", "gpt-4-32k"],
    },
)

print(config_list[0]["model"])

/home/lijiang1/anaconda3/envs/autogen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


gpt-3.5-turbo


It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well).

The config list looks like the following:
```python
config_list = [
    {
        "model": "gpt-4",
        "api_key": "<your OpenAI API key>",
    },  # OpenAI API endpoint for gpt-4
    {
        "engine": "gpt-35-turbo", 
        "model": "gpt-3.5-turbo",
        "api_base": "<your Azure OpenAI API base>", 
        "api_type": "azure", 
        "api_version": "2023-08-01-preview", # must be newer than 2023-07-01-preview to use functions
        "api_key": "<your Azure OpenAI API key>"
    }
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways you prefer, e.g., loading from a YAML file.

## Construct Agents

In [3]:
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb

llm_config = {
    "request_timeout": 60,
    "seed": 42,
    "config_list": config_list,
}

autogen.ChatCompletion.start_logging()
termination_msg = lambda x: "TERMINATE" == x.get("content", "")[-9:].upper()

boss = autogen.UserProxyAgent(
    name="Boss",
    is_termination_msg=termination_msg,
    system_message="The boss who ask questions and give tasks.",
    code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
    human_input_mode="TERMINATE"
)

boss_aid = RetrieveUserProxyAgent(
    name="Boss_Assistant",
    is_termination_msg=termination_msg,
    system_message="Assistant who has extra content retrieval power for solving difficult problems.",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "code",
        "docs_path": "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Integrate%20-%20Spark.md",
        "chunk_token_size": 1000,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "collection_name": "groupchat",
        "get_or_create": True,
    },
    code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
)

coder = RetrieveAssistantAgent(
    name="Senior_Python_Engineer",
    is_termination_msg=termination_msg,
    system_message="You are a senior python engineer. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

pm = autogen.AssistantAgent(
    name="Product_Manager",
    is_termination_msg=termination_msg,
    system_message="You are a product manager. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

reviewer = autogen.AssistantAgent(
    name="Code_Reviewer",
    is_termination_msg=termination_msg,
    system_message="You are a code reviewer. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

PROBLEM = "How to use spark for parallel training in FLAML? Give me sample code."

def rag_chat():
    groupchat = autogen.GroupChat(
        agents=[boss_aid, coder, pm, reviewer], messages=[], max_round=12
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss_aid as this is the user proxy agent.
    boss_aid.initiate_chat(
        manager,
        problem=PROBLEM,
        n_results=3,
    )


def norag_chat():
    groupchat = autogen.GroupChat(
        agents=[boss, coder, pm, reviewer], messages=[], max_round=12
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss as this is the user proxy agent.
    boss.initiate_chat(
        manager,
        message=PROBLEM,
    )


def call_rag_chat():
    # In this case, we will have multiple user proxy agents and we don't initiate the chat
    # with RAG user proxy agent.
    # In order to use RAG user proxy agent, we need to wrap RAG agents in a function and call
    # it from other agents.
    def retrieve_content(message, n_results=3):
        boss_aid.n_results = n_results  # Set the number of results to be retrieved.
        # Check if we need to update the context.
        update_context_case1, update_context_case2 = boss_aid._check_update_context(message)
        if (update_context_case1 or update_context_case2) and boss_aid.update_context:
            boss_aid.problem = message if not hasattr(boss_aid, "problem") else boss_aid.problem
            _, ret_msg = boss_aid._generate_retrieve_user_reply(message)
        else:
            ret_msg = boss_aid.generate_init_message(message, n_results=n_results)
        return ret_msg if ret_msg else message
    
    boss_aid._code_execution_config = False # Disable code execution for boss_aid since it only retrieves content.
    boss_aid.human_input_mode = "NEVER" # Disable human input for boss_aid since it only retrieves content.
    
    llm_config = {
        "functions": [
            {
                "name": "retrieve_content",
                "description": "retrieve content for code generation and question answering.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "The message to query for content.",
                        }
                    },
                    "required": ["message"],
                },
            },
        ],
        "config_list": config_list,
        "request_timeout": 60,
        "seed": 42,
    }
    
    coder.llm_config.update(llm_config)
    pm.llm_config.update(llm_config)
    reviewer.llm_config.update(llm_config)
    boss.register_function(
        function_map={
            "retrieve_content": retrieve_content,
        }
    )

    groupchat = autogen.GroupChat(
        agents=[boss, coder, pm, reviewer], messages=[], max_round=12
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss as this is the user proxy agent.
    boss.initiate_chat(
        manager,
        message=PROBLEM,
    )

## Start Chat

### UserProxyAgent doesn't get the correct code
[FLAML](https://github.com/microsoft/FLAML) was open sourced in 2020, so ChatGPT is familiar with it. However, Spark-related APIs were added in 2022, so they were not in ChatGPT's training data. As a result, we end up with invalid code.

In [4]:
norag_chat()

Boss (to chat_manager):

How to use spark for parallel training in FLAML? Give me sample code.

--------------------------------------------------------------------------------
Senior_Python_Engineer (to chat_manager):

Sure, here is a sample code for parallel training using FLAML and Spark:

```python
from flaml import AutoML
from pyspark.sql import SparkSession

# initialize spark session
spark = SparkSession.builder \
        .appName("FLAML-Spark Demo") \
        .getOrCreate()

# read data into spark dataframe
df = spark.read.format("csv").option("header", "true").load("hdfs://<hdfs-path>/data.csv")

# convert spark dataframe to pandas dataframe
train_data = df.toPandas()

# define automl settings
settings = {
    "time_budget": 300,  # in seconds
    "task": "classification",
    "log_file_name": "flaml.log",
    "model_history": True,
    "grace_period": 30,
    "n_jobs": -1,
}

# initialize spark context
sc = spark.sparkContext

# run automl on spark context
automl = AutoML()
b

Boss (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Traceback (most recent call last):
  File "", line 10, in <module>
    df = spark.read.format("csv").option("header", "true").load("hdfs://<hdfs-path>/data.csv")
  File "/datadrive/spark/spark33/python/pyspark/sql/readwriter.py", line 177, in load
    return self._df(self._jreader.load(path))
  File "/datadrive/spark/spark33/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1321, in __call__
  File "/datadrive/spark/spark33/python/pyspark/sql/utils.py", line 190, in deco
    return f(*a, **kw)
  File "/datadrive/spark/spark33/python/lib/py4j-0.10.9.5-src.zip/py4j/protocol.py", line 326, in get_return_value
py4j.protocol.Py4JJavaError: An error occurred while calling o27.load.
: java.io.IOException: Incomplete HDFS URI, no host: hdfs://%3Chdfs-path%3E/data.csv
	at org.apache.hado

Boss (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 1, in <module>
    df = spark.read.format("csv").option("header", "true").load("hdfs://<hdfs-hostname>:<hdfs-port>/<path-to-data-file>/data.csv")
NameError: name 'spark' is not defined


--------------------------------------------------------------------------------
Senior_Python_Engineer (to chat_manager):

The error message indicates that the Spark session might not have been initialized before reading the data file from HDFS. Please make sure that you have properly initialized the Spark session before running the code.

Here is the updated code with the correct HDFS path and Spark session:

```python
from flaml import AutoML
from pyspark.sql import SparkSession

# initialize spark session
spark = SparkSession.builder \
        .appName("FLAML-Spark Demo") \
        .getOrCreate()

# read data into spark dataframe
df = spark.read.format("csv").option("header", "tr

Boss (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Traceback (most recent call last):
  File "", line 10, in <module>
    df = spark.read.format("csv").option("header", "true").load("hdfs://<hdfs-hostname>:<hdfs-port>/<path-to-data-file>/data.csv")
  File "/datadrive/spark/spark33/python/pyspark/sql/readwriter.py", line 177, in load
    return self._df(self._jreader.load(path))
  File "/datadrive/spark/spark33/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1321, in __call__
  File "/datadrive/spark/spark33/python/pyspark/sql/utils.py", line 190, in deco
    return f(*a, **kw)
  File "/datadrive/spark/spark33/python/lib/py4j-0.10.9.5-src.zip/py4j/protocol.py", line 326, in get_return_value
py4j.protocol.Py4JJavaError: An error occurred while calling o27.load.
: java.io.IOException: Incomplete HDFS URI, no host: hdfs://%3Chdfs-

Boss (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 1, in <module>
    df = spark.read.format("csv").option("header", "true").load("hdfs://<hdfs-hostname>:<hdfs-port>/<path-to-data-file>/data.csv")
NameError: name 'spark' is not defined


--------------------------------------------------------------------------------
exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 1, in <module>
    df = spark.read.format("csv").option("header", "true").load("hdfs://<hdfs-hostname>:<hdfs-port>/<path-to-data-file>/data.csv")
NameError: name 'spark' is not defined


--------------------------------------------------------------------------------


### RetrieveUserProxyAgent get the correct code
Since RetrieveUserProxyAgent can perform retrieval-augmented generation based on the given documentation file, ChatGPT can generate the correct code for us!

In [5]:
rag_chat()
# type exit to terminate the chat

Trying to create collection.


INFO:autogen.retrieve_utils:Found 2 chunks.


doc_ids:  [['doc_0', 'doc_1', 'doc_4']]
Adding doc_id doc_0 to context.
Adding doc_id doc_1 to context.
Adding doc_id doc_4 to context.
Boss_Assistant (to chat_manager):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: How to use spark for parallel training in FLAML? Give me sample code.

Context is: # Integrate - Spark

FLAML has integrated Spark for distributed training. There are two main aspects of integration with Spark:
- Use Spark ML estimators for AutoML.
- Use Spark to run training in parallel spark jobs.

## Spark M

### Call RetrieveUserProxyAgent while init chat with another user proxy agent
Sometimes, we want to use RetrieveUserProxyAgent in the group chat, but we don't want to init the chat with RetrieveUserProxyAgent.

In [7]:
call_rag_chat()

Boss (to chat_manager):

How to use spark for parallel training in FLAML? Give me sample code.

--------------------------------------------------------------------------------


InvalidRequestError: Unrecognized request argument supplied: functions